# 1. Information about the submission

## 1.1 Name and number of the assignment

Assignment 3. Text Detoxification.

## 1.2 Student name

Bogdan Alexandrov

## 1.3 Codalab user ID

bogchamp

## 1.4 Additional comments

***Enter here** any additional comments which you would like to communicate to a TA who is going to grade this work not related to the content of your submission.*

# 2. Technical Report

*Use Section 2 to describe results of your experiments as you would do writing a paper about your results. DO NOT insert code in this part. Only insert plots and tables summarizing results as needed. Use formulas if needed do described your methodology. The code is provided in Section 3.*

## 2.1 Methodology 

For solving this problem I used T5 model, which is very popular model for solving such kind of tasks. As base model I used "sberbank-ai/ruT5-base", trained on russian corpus of texts. None additional preprocessing was performed, as augmentations I used random deletion of words and and random swap.

As optimizer used Adam with standart parameters, number of epochs - 5.


## 2.2 Discussion of results

***Enter here** a discussion of results and a summary of the experiment. Here we want to see the final table with comparison of the baseline and all tried approaches you decided to report. Even if some method did not bring you to the top of the leaderboard, you should nevertheless indicate this result and a discussion, why, in your opinion, some approach worked and another failed. Interesting findings in the discussion will be a plus.*

Method | Score 
--- | --- 
Baseline | 0.41 
My T5 | 0.48
My T5 with augments | 0.467

As we can see, simple T5 can beat baseline, if we train it more than 3 epochs(less will give worth result). Also only models pretrained on russian corpus should be used, otherwise score will be low(about 0.18 for t5-base). Augmentations didn't give me any improves. Probably, they don't really suit to this problem, or we need to change hyperparameters of augmentations. The only thing I found - the more epochs I train model, the bigger become score.


# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [ ]:
!pip install gdown -q
!pip install transformers sequential sentencepiece datasets -q
!pip install --upgrade accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 8.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
from tqdm import tqdm
import gc
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset as torchDataset
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import random

## 3.2 Download the data

In [ ]:
! wget https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/train.tsv -q
! wget https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/test.tsv -q
! wget https://raw.githubusercontent.com/s-nlp/russe_detox_2022/main/data/input/dev.tsv -q

## 3.3 Preprocessing 

In [ ]:
import pandas as pd

train = pd.read_csv('train.tsv', sep='\t', index_col=0)
train.head()

,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
index,,,,
0,"и,чё,блядь где этот херой был до этого со свои...","Ну и где этот герой был,со своими доказательст...",Где этот герой был до этого со своими доказате...,"и,где этот герой был до этого со своими доказа..."
1,"О, а есть деанон этого петуха?","О, а есть деанон",NaN,NaN
2,"херну всякую пишут,из-за этого лайка.долбоебизм.","Чушь всякую пишут, из- за этого лайка.","Ерунду всякую пишут,из-за этого лайка.",NaN
3,из за таких пидоров мы и страдаем,из за таких плохих людей мы и страдаем,Из-за таких людей мы и страдаем,из за таких как он мы и страдаем
4,гондон путинский он а не артист,"Человек Путина он, а не артист",NaN,NaN


In [ ]:
dev = pd.read_csv('dev.tsv', sep='\t')
dev.head()

,toxic_comment,neutral_comment1,neutral_comment2,neutral_comment3
0,пиздеж! температуры горения хватит чтобы её ра...,Враньё! Температуры горения хватит чтобы ее ра...,"неправда,температуры горения хватит чтобы расп...",Враньё! Температуры горения хватит на чтобы её...
1,а ты чмо там был.ты вообще служил.гандон,А ты там был? Ты вообще служил?,NaN,NaN
2,пиздабол ---- а сам где кормишься ?,а сам где кормишься ?,NaN,NaN
3,"этому сайту я давно не доверяю, пишут разную х...","Этому сайту давно не доверяю, пишут всякую ерунду",NaN,NaN
4,у меня сегодня подобный звонок был. достали го...,у меня сегодня подобный звонок был. Достали.,"У меня сегодня подобный звонок был,надоели",NaN


In [ ]:
test = pd.read_csv('test.tsv', sep='\t')
test.head()

,toxic_comment
0,какой дебил эту придумывает
1,В такой ситуации виноваты пидарасы из Ростелек...
2,"актёр может и не плохой,но как человек - хуйло..."
3,мочите всех кто нарушает общественный порядок
4,такие же твари и привели этих тварей


In [ ]:
df_train_toxic = []
df_train_neutral = []

for index, row in train.iterrows():
    references = row[['neutral_comment1', 'neutral_comment2', 'neutral_comment3']].tolist()
    
    for reference in references:

        if not (reference is np.nan):
            df_train_toxic.append(row['toxic_comment'])
            df_train_neutral.append(reference)
        else:
            break

In [ ]:
df = pd.DataFrame({
    'toxic_comment': df_train_toxic,
    'neutral_comment1': df_train_neutral
})

train = df.sample(frac=1)

In [ ]:
df_train_toxic = []
df_train_neutral = []

for index, row in dev.iterrows():
    references = row[['neutral_comment1', 'neutral_comment2', 'neutral_comment3']].tolist()
    
    for reference in references:

        if not (reference is np.nan):
            df_train_toxic.append(row['toxic_comment'])
            df_train_neutral.append(reference)
        else:
            break

In [ ]:
df = pd.DataFrame({
    'toxic_comment': df_train_toxic,
    'neutral_comment1': df_train_neutral
})

dev = df

## 3.4 Dataset prepairing

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("sberbank-ai/ruT5-base")

In [ ]:
max_source_length = 128
max_target_length = 128

data_config = {'add_special_tokens':True,
                'max_length': max_source_length,
                'truncation':True,
                'return_token_type_ids':False,
                'padding':'max_length',
                'return_attention_mask':True,
                'return_tensors':'pt',
                'verbose':False}

target_config = {'add_special_tokens':True,
                'max_length': max_target_length,
                'truncation':True,
                'return_token_type_ids':False,
                'padding':'max_length',
                'return_attention_mask':True,
                'return_tensors':'pt',
                'verbose':False}

In [ ]:
def random_deletion(sentence, p=0.3):
    """Randomly delete words from the sentence with a given probability"""
    words = sentence.split()
    new_sentence = [word for word in words if random.uniform(0, 1) > p]
    return ' '.join(new_sentence)

def random_swap(sentence, n=2):
    """Randomly swap two adjacent words in the sentence"""
    words = sentence.split()
    new_sentence = sentence.split()
    for i in range(n):
        if len(words) > 1:
            idx_1, idx_2 = random.sample(range(len(words)-1), 2)
            new_sentence[idx_1], new_sentence[idx_2] = new_sentence[idx_2], new_sentence[idx_1]
    return ' '.join(new_sentence)

def apply_augmentation(sentence_tokenized, attention_mask, p=0.125):
    apply_deletion = np.random.binomial(1, p, size=1)
    apply_swap = np.random.binomial(1, p, size=1)
    
    if not (apply_deletion[0] or apply_swap[0]):
        return sentence_tokenized, attention_mask
        
    sentence = tokenizer.decode(sentence_tokenized, skip_special_tokens=True)

    if apply_deletion[0]:
        sentence = random_deletion(sentence)
    if apply_swap[0]:
        sentence = random_deletion(sentence)
    sentence = tokenizer(sentence, **data_config)

    return sentence.input_ids.squeeze(), sentence.attention_mask.squeeze()

class T5Dataset(torchDataset):
    def __init__(self, data, tokenizer, data_config, target_config, do_augmentation=False):
        super().__init__()
        self.source_tokens = tokenizer(data.toxic_comment.to_list(), **data_config)
        self.target_tokens = tokenizer(data.neutral_comment1.to_list(), **target_config)
        self.length = len(self.source_tokens['input_ids'])
        self.do_augmentation = do_augmentation

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        input_ids = self.source_tokens['input_ids'][index]
        attention_mask = self.source_tokens['attention_mask'][index]
        label = self.target_tokens['input_ids'][index]
        if self.do_augmentation:
            input_ids, attention_mask = apply_augmentation(input_ids, attention_mask)
        
        return {
          'input_ids':input_ids,
          'attention_mask':attention_mask,
          'labels': label
        }

In [ ]:
train_dataset = T5Dataset(train, tokenizer, data_config, target_config, do_augmentation=False)
val_dataset = T5Dataset(dev, tokenizer, data_config, target_config)

## 3.5 Model preparing

In [ ]:
num_epochs = 5
batch_size = 16

args = Seq2SeqTrainingArguments(
    output_dir="output",
    evaluation_strategy = "steps",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    logging_strategy='steps',
    logging_steps=500,
    metric_for_best_model='loss',
    skip_memory_metrics = False,
    dataloader_num_workers = 2
)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("sberbank-ai/ruT5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

## 3.6 Model training & evaluation

In [ ]:
trainer.train()
gc.collect()
torch.cuda.empty_cache()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.533800,0.168199
1000,0.199900,0.150606
1500,0.178900,0.148834
2000,0.164900,0.145411
2500,0.159400,0.144393
3000,0.149800,0.142716


In [ ]:
trainer.evaluate()

{'eval_loss': 0.14310555160045624,
 'eval_runtime': 8.0501,
 'eval_samples_per_second': 99.377,
 'eval_steps_per_second': 6.211,
 'epoch': 5.0,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 69632,
 'eval_mem_gpu_peaked_delta': 851252736}

## 3.7 Model inference

In [ ]:
def paraphrase(
    text,
    model,
    tokenizer,
    n=None,
    max_length="auto",
    beams=5,
):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        # forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang],
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [ ]:
preds = []
for text in tqdm(dev.toxic_comment):
    preds.append(paraphrase(text, model, tokenizer))

100%|██████████| 800/800 [04:23<00:00,  3.03it/s]


In [ ]:
with open(f"predictions.txt", "w") as f:
    f.write("\n".join(preds))

In [ ]:
preds[:10]

['Кто эту придумывает? Он её придумывает',
 'В такой ситуации виноваты люди из Ростелекома у которых даже кошка может купить фильм с пульта',
 'актёр может и не плохой,но как человек - плохое.',
 'мочите всех кто нарушает общественный порядок.',
 'Такие же люди и привели этих людей.',
 'А зачем тогда ты здесь это писал?',
 'Главный повар из Миннеаполиса сварщик из бейрута президент минска из d',
 'Начни со своих нехороших людей.',
 'Дайте уже пожить создать семью отдал 35 лет жизни,кормил власти 30 лет мой отец 60 лет хватит',
 'А ты что 41 год помнишь? Сколько ей денег заплатили, чтобы она такую чушь несла.']

In [ ]:
test['neutral_comment'] = preds

In [ ]:
test.to_csv('pred.csv')